In [ ]:
# @title
from IPython.display import display, HTML

display(HTML("""
<script>
const firstCell = document.querySelector('.cell.code_cell');
if (firstCell) {
  firstCell.querySelector('.input').style.pointerEvents = 'none';
  firstCell.querySelector('.input').style.opacity = '0.5';
}
</script>
"""))

html = """
<div style="display:flex; flex-direction:column; align-items:center; text-align:center; gap:12px; padding:8px;">
  <h1 style="margin:0;">👋 Welcome to <span style="color:#1E88E5;">Algopath Coding Academy</span>!</h1>

  <img src="https://raw.githubusercontent.com/sshariqali/mnist_pretrained_model/main/algopath_logo.jpg"
       alt="Algopath Coding Academy Logo"
       width="400"
       style="border-radius:15px; box-shadow:0 4px 12px rgba(0,0,0,0.2); max-width:100%; height:auto;" />

  <p style="font-size:16px; margin:0;">
    <em>Empowering young minds to think creatively, code intelligently, and build the future with AI.</em>
  </p>
</div>
"""

display(HTML(html))

### Importing the libraries

TO DO:
- Add Token Embedding Layer
- Add Positional Embedding Layer
- Add Linear Layer

In [1]:
import torch

### Defining the Hyperparameters

In [16]:
batch_size = 32 # how many independent sequences to process in parallel
block_size = 8 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32

### Loading the Dataset

In [17]:
# Load the tiny shakespeare dataset
dataset = "tiny_shakespeare.txt"

# Load the dataset into a string
with open(dataset, "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

### Defining the Encoder and Decoder

In [18]:
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

### Splitting the Data

In [19]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

### Data Loading

In [20]:
torch.manual_seed(1337)

def get_batch(split):
    # generate a small batch of input-target pairs
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

### Loss Function

In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Single Head

In [22]:
class Head(torch.nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = torch.nn.Linear(n_embed, head_size, bias=False)
        self.query = torch.nn.Linear(n_embed, head_size, bias=False)
        self.value = torch.nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = torch.nn.functional.softmax(wei, dim=-1)
        out = wei @ v
        return out

### Transformer Model

In [ ]:
torch.manual_seed(1337)

class Transformer(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = torch.nn.Embedding(block_size, n_embed)
        self.sa_head = Head(n_embed)
        self.lm_head = torch.nn.Linear(n_embed, vocab_size)

    def forward(self, idx, y = None):
        # idx is (B, T) tensor of indices.
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, n_embed)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb
        x = self.sa_head(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        # Loss
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = torch.functional.F.cross_entropy(logits, y)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) tensor of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = torch.nn.functional.softmax(logits, dim = -1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx

In [ ]:
model = Transformer()
model = model.to(device)

### Training the Model

In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [26]:
for i in range(max_iters):
    
    if i % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)

    # perform backpropagation
    loss.backward()

    # update the weights
    optimizer.step()

    # zero the gradients
    optimizer.zero_grad(set_to_none = True)

step 0: train loss 4.2000, val loss 4.2047
step 300: train loss 2.9418, val loss 2.9574
step 600: train loss 2.6309, val loss 2.6456
step 900: train loss 2.5385, val loss 2.5451
step 1200: train loss 2.5040, val loss 2.5033
step 1500: train loss 2.4695, val loss 2.4808
step 1800: train loss 2.4520, val loss 2.4641
step 2100: train loss 2.4385, val loss 2.4422
step 2400: train loss 2.4241, val loss 2.4419
step 2700: train loss 2.4122, val loss 2.4340
step 3000: train loss 2.4227, val loss 2.4241
step 3300: train loss 2.4159, val loss 2.4231
step 3600: train loss 2.4055, val loss 2.4177
step 3900: train loss 2.4023, val loss 2.4179
step 4200: train loss 2.3958, val loss 2.4302
step 4500: train loss 2.3888, val loss 2.4024
step 4800: train loss 2.3838, val loss 2.4043
step 5100: train loss 2.3852, val loss 2.4064
step 5400: train loss 2.3925, val loss 2.4025
step 5700: train loss 2.3870, val loss 2.3913
step 6000: train loss 2.3840, val loss 2.3910
step 6300: train loss 2.3755, val loss 2

In [27]:
context = torch.zeros((1, 1), dtype = torch.long) # Since idx 0 is a new line character
out = model.generate(context, max_new_tokens = 200)
print(decode(out[0].tolist()))


RIV
spr
Ans Ste'd, Los angdan herl I my fary bothaper oord maly ond by haur to! ade bind I chay ne, witst gh ithe,
Senin I: at tonk, ble,' I mow hpero' w kins Beve thul.

O! rcacrhfo isun.
 Suce th me
